**BERT**

In [1]:
!pip install -U sentence-transformers
!pip install torch
!pip install tensorflow
!pip install transformers==4.12.2
!pip install spacy

  Using cached transformers-4.12.2-py3-none-any.whl (3.1 MB)
  Obtaining dependency information for sacremoses from https://files.pythonhosted.org/packages/0b/f0/89ee2bc9da434bd78464f288fdb346bc2932f2ee80a90b2a4bbbac262c74/sacremoses-0.1.1-py3-none-any.whl.metadata
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers
      creating build\lib.win-amd64-cpython-311\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\models
      creating build\lib.win-amd64-cpython-311\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\decoders
      creating build\lib.win-amd64-cpython-311\tokenizers\normalizers
      copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-311\tokenizers\normalizers
      creating build\lib.

In [2]:
import json
import torch
from transformers import BertTokenizer, BertModel
import spacy
from difflib import SequenceMatcher

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)


user_question = input("Ask a question: ")

# Tokenize and encode the user's question
user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

# Calculate cosine similarity between the user's question and all questions in the JSON file
similarities = []
for item in data:
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)

    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding)
    similarities.append(cosine_sim.item())

max_index = similarities.index(max(similarities))
most_similar_answer = data[max_index]['answer']
print("Most relevant answer:", most_similar_answer)


Most relevant answer: When Mane will share confidential information with another person or entity.  Also, when someone will observe or learn about Mane's confidential information. Additionally, when any of the parties (MANE and Counterparty) wish to disclose confidential information to each other.


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)

user_question = input("Ask a question: ")
user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)
user_question = input("Ask a question: ")
threshold = 0.8
similarities = []
for i, item in enumerate(data):
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)

    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding).item()

    if cosine_sim > threshold:
        print(f"Question {i + 1}: {item['question']}")
        print(f"Cosine Similarity Score: {cosine_sim}")
        print()

        similarities.append(cosine_sim)

if similarities:
    sorted_similarities = sorted(similarities, reverse=True)
    max_index = similarities.index(sorted_similarities[0])
    most_similar_answer = data[max_index]['answer']

    print("Most relevant answer:", most_similar_answer)
    print("Cosine Similarity Score with most relevant question:", sorted_similarities[0])

    # Check the difference between the top two similarity scores
    score_difference = sorted_similarities[0] - sorted_similarities[1]

    # Set a threshold for prompting the user for more specificity
    specificity_threshold = 0.05

    if score_difference < specificity_threshold:
        print("The similarity scores are very close. Please be more specific in your question.")
else:
    print("No relevant questions found above the threshold.")


Question 1: What is the difference between a Commercial Representative and a Distributor?
Cosine Similarity Score: 0.9281597137451172

Question 2: What is the difference between a Commercial Representative and an Agent?
Cosine Similarity Score: 0.9324036836624146

Most relevant answer: An Agent is entitled to negotiate selling prices on behalf of the company it represents, while a Commercial Representative is not. MANE does not appoint Agents, only Commercial Representatives.
Cosine Similarity Score with most relevant question: 0.9324036836624146
The similarity scores are very close. Please be more specific in your question.


**Bert base model**

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

nlp = spacy.load("en_core_web_sm")

with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)
user_question = input("Ask a question: ")

user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

similarities = []
for i, item in enumerate(data):
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)

    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding).item()
    similarities.append(cosine_sim)

top_similar_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:2]

for idx in top_similar_indices:
    print(f"Question {idx + 1}: {data[idx]['question']}")
    print(f"Cosine Similarity Score: {similarities[idx]}")

user_response = input("Do you want to clarify your question for better results? (yes/no): ")

if user_response.lower() == 'yes':
    prev_question = data[top_similar_indices[0]]['question']
    refined_question = input("Please refine your question: ")

    refined_tokens = nlp(refined_question)
    refined_lemmas = " ".join([token.lemma_ for token in refined_tokens])

    prev_tokens = nlp(prev_question)
    prev_lemmas = " ".join([token.lemma_ for token in prev_tokens])

    lemmatized_question = f"{refined_lemmas} {prev_lemmas}"
    lemmatized_tokens = tokenizer.encode(lemmatized_question, truncation=True, max_length=512, return_tensors='pt')
    lemmatized_embedding = model(input_ids=lemmatized_tokens).last_hidden_state.mean(dim=1)

    prev_tokens = tokenizer.encode(prev_question, truncation=True, max_length=512, return_tensors='pt')
    prev_embedding = model(input_ids=prev_tokens).last_hidden_state.mean(dim=1)
    lemmatized_cosine_sim = torch.nn.functional.cosine_similarity(lemmatized_embedding, prev_embedding).item()

    print(f"Cosine Similarity Score with refined question: {lemmatized_cosine_sim}")
    if lemmatized_cosine_sim > 0.8:
        print("Most relevant answer:", data[top_similar_indices[0]]['answer'])
    else:
        print("No relevant answer found for the refined question.")


Question 12: What is the purpose of the commercial contract review-process?
Cosine Similarity Score: 0.789609432220459
Question 78: What BI means in a CDA?
Cosine Similarity Score: 0.780975341796875
Cosine Similarity Score with refined question: 0.8781399130821228
Most relevant answer: The contract review process provides for the conditions of verification and modifications of agreement proposals concerning the Mane Group. The purpose of this process is to make sure Mane actually wants and can comply with the terms and conditions offered/submitted to it, and that such terms and conditions are compliant with Mane's policies and operating methods. The Legal & IP Department proceeds with the final review of the concerned document based on the feedbacks of each concerned department at Mane.


In [5]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Load spaCy for lemmatization
nlp = spacy.load("en_core_web_sm")

# Load your FAQ data
with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)

# Get user question
user_question = input("Ask a question: ")

# Tokenize and embed user question
user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

# Calculate cosine similarity with each FAQ question
similarities = []
for i, item in enumerate(data):
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)
    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding).item()
    similarities.append(cosine_sim)

# Sort and get top similar indices
top_similar_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)

# Function to handle responses based on cosine similarity
def handle_response(threshold_low, threshold_high, top_indices):
    for idx in top_indices:
        print(f"Question {idx + 1}: {data[idx]['question']}")
        print(f"Answer: {data[idx]['answer']}")
        print(f"Cosine Similarity Score: {similarities[idx]}")
        print()

# Set thresholds and handle responses
if similarities[top_similar_indices[0]] > 0.8:
    # High similarity, return the first answer
    handle_response(0.8, 1.0, top_similar_indices[:1])
elif 0.5 < similarities[top_similar_indices[0]] <= 0.8:
    # Moderate similarity, check the next 3 or 4 answers
    handle_response(0.5, 0.8, top_similar_indices[:4])
    # Check for very close scores and suggest refining the question
    if similarities[top_similar_indices[0]] - similarities[top_similar_indices[1]] < 0.1:
        user_response = input("Your question seems close to multiple FAQs. Do you want to refine your question for better results? (yes/no): ")
        if user_response.lower() == 'yes':
            prev_question = data[top_similar_indices[0]]['question']
            refined_question = input("Please refine your question: ")

            refined_tokens = nlp(refined_question)
            refined_lemmas = " ".join([token.lemma_ for token in refined_tokens])

            prev_tokens = nlp(prev_question)
            prev_lemmas = " ".join([token.lemma_ for token in prev_tokens])

            lemmatized_question = f"{refined_lemmas} {prev_lemmas}"
            lemmatized_tokens = tokenizer.encode(lemmatized_question, truncation=True, max_length=512, return_tensors='pt')
            lemmatized_embedding = model(input_ids=lemmatized_tokens).last_hidden_state.mean(dim=1)

            prev_tokens = tokenizer.encode(prev_question, truncation=True, max_length=512, return_tensors='pt')
            prev_embedding = model(input_ids=prev_tokens).last_hidden_state.mean(dim=1)
            lemmatized_cosine_sim = torch.nn.functional.cosine_similarity(lemmatized_embedding, prev_embedding).item()

            print(f"Cosine Similarity Score with refined question: {lemmatized_cosine_sim}")
            
            # Historical Scenario: If best distance score is less than 0.4
            if lemmatized_cosine_sim < 0.4:
                # Run LLM in inference GPT-3 (implementation required)
                print("Running LLM in inference GPT-3...")
            else:
                # Return the most relevant answer for the refined question
                handle_response(0.8, 1.0, top_similar_indices[:1])
        else:
            print("No relevant answer found for the refined question.")
else:
    print("No relevant answer found for the original question.")


No relevant answer found for the original question.




**Bert large model**

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained('bert-large-uncased')

nlp = spacy.load("en_core_web_sm")

In [11]:
with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)
user_question = input("Ask a question: ")

user_question_tokens = tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

similarities = []
for i, item in enumerate(data):
    question_tokens = tokenizer.encode(item['question'], truncation=True, max_length=512, return_tensors='pt')
    question_embedding = model(input_ids=question_tokens).last_hidden_state.mean(dim=1)
    cosine_sim = torch.nn.functional.cosine_similarity(user_question_embedding, question_embedding, dim=1).item()
    similarities.append(cosine_sim)
top_similar_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:2]
for idx in top_similar_indices:
    print(f"Question {idx + 1}: {data[idx]['question']}")
    print(f"Cosine Similarity Score: {similarities[idx]}")

user_response = input("Do you want to clarify your question for better results? (yes/no): ")

if user_response.lower() == 'yes':
    prev_question = data[top_similar_indices[0]]['question']
    refined_question = input("Please refine your question: ")
    refined_tokens = nlp(refined_question)
    refined_lemmas = " ".join([token.lemma_ for token in refined_tokens])

    prev_tokens = nlp(prev_question)
    prev_lemmas = " ".join([token.lemma_ for token in prev_tokens])

    lemmatized_question = f"{refined_lemmas} {prev_lemmas}"
    lemmatized_tokens = tokenizer.encode(lemmatized_question, truncation=True, max_length=512, return_tensors='pt')
    lemmatized_embedding = model(input_ids=lemmatized_tokens).last_hidden_state.mean(dim=1)

    prev_tokens = tokenizer.encode(prev_question, truncation=True, max_length=512, return_tensors='pt')
    prev_embedding = model(input_ids=prev_tokens).last_hidden_state.mean(dim=1)
    lemmatized_cosine_sim = torch.nn.functional.cosine_similarity(lemmatized_embedding, prev_embedding, dim=1).item()

    print(f"Cosine Similarity Score with refined question: {lemmatized_cosine_sim}")
    if lemmatized_cosine_sim > 0.8:
        print("Most relevant answer:", data[top_similar_indices[0]]['answer'])
    else:
        print("No relevant answer found for the refined question.")


Question 36: When is a CDA needed?
Cosine Similarity Score: 0.9674476385116577
Question 50: Is a CDA needed for a facility visit?
Cosine Similarity Score: 0.8730107545852661


In [9]:
from transformers import BertTokenizer, BertModel, GPT2LMHeadModel, GPT2Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

lm_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
lm_model = GPT2LMHeadModel.from_pretrained('gpt2')

nlp = spacy.load("en_core_web_sm")

def cosine_similarity_score(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2).item()

def get_top_similar_indices(data, user_question_embedding, num_results=2):
    similarities = [cosine_similarity_score(user_question_embedding, model(input_ids=bert_tokenizer.encode(data[idx]['question'], truncation=True, max_length=512, return_tensors='pt')).last_hidden_state.mean(dim=1)) for idx in range(len(data))]

    top_similar_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:num_results]
    return top_similar_indices, similarities

def print_top_similar_questions(data, top_similar_indices, similarities):
    for idx in top_similar_indices:
        print(f"Question {idx + 1}: {data[idx]['question']}")
        print(f"Cosine Similarity Score: {similarities[idx]}")

def handle_response_threshold(similarity_scores, data, top_similar_indices):
    if similarity_scores[0] < 0.5:
        print("Most relevant answer:", data[top_similar_indices[0]]['answer'])
    elif 0.5 <= similarity_scores[0] < 0.8:
        next_answers_indices = top_similar_indices[1:5]  
        print_top_similar_questions(data, next_answers_indices, similarities)
    else:
        print("Scores are very close. Suggest refining the question.")

def handle_lack_of_context(data, top_similar_indices):
    user_response = input("Do you want to clarify your question for better results? (yes/no): ")
    if user_response.lower() == 'yes':
        prev_question = data[top_similar_indices[0]]['question']
        refined_question = input("Please refine your question: ")

        refined_tokens = nlp(refined_question)
        refined_lemmas = " ".join([token.lemma_ for token in refined_tokens])

        prev_tokens = nlp(prev_question)
        prev_lemmas = " ".join([token.lemma_ for token in prev_tokens])

        lemmatized_question = f"{refined_lemmas} {prev_lemmas}"
        lemmatized_tokens = bert_tokenizer.encode(lemmatized_question, truncation=True, max_length=512, return_tensors='pt')
        lemmatized_embedding = bert_model(input_ids=lemmatized_tokens).last_hidden_state.mean(dim=1)

        prev_tokens = bert_tokenizer.encode(prev_question, truncation=True, max_length=512, return_tensors='pt')
        prev_embedding = bert_model(input_ids=prev_tokens).last_hidden_state.mean(dim=1)
        lemmatized_cosine_sim = cosine_similarity_score(lemmatized_embedding, prev_embedding)

        print(f"Cosine Similarity Score with refined question: {lemmatized_cosine_sim}")
        if lemmatized_cosine_sim > 0.8:
            print("Most relevant answer:", data[top_similar_indices[0]]['answer'])
        else:
            print("No relevant answer found for the refined question.")

def handle_gpt2_response(data, top_similar_indices, similarities):
    best_distance_score = similarities[0]
    if best_distance_score < 0.5:
        user_question = input("Ask a follow-up question for historical scenario: ")
        # Run GPT-2 for inference with the user's question
        lm_input = lm_tokenizer.encode(user_question, return_tensors='pt')
        lm_output = lm_model.generate(lm_input, max_length=100, pad_token_id=lm_tokenizer.eos_token_id, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
        lm_answer = lm_tokenizer.decode(lm_output[0], skip_special_tokens=True)
        print("GPT-2 Answer:", lm_answer)
    else:
        print("No need for GPT-2 in this scenario.")

with open('modified_faq_data.json', 'r') as json_file:
    data = json.load(json_file)

user_question = input("Ask a question: ")
user_question_tokens = bert_tokenizer.encode(user_question, truncation=True, max_length=512, return_tensors='pt')
user_question_embedding = bert_model(input_ids=user_question_tokens).last_hidden_state.mean(dim=1)

top_similar_indices, similarities = get_top_similar_indices(data, user_question_embedding, num_results=2)
print_top_similar_questions(data, top_similar_indices, similarities)
handle_response_threshold(similarities, data, top_similar_indices)
handle_lack_of_context(data, top_similar_indices)
historical_scenario(data, top_similar_indices)
handle_gpt2_response(data, top_similar_indices, similarities)


Question 69: Where can I find the contract review form?
Cosine Similarity Score: 0.5336938500404358
Question 19: What are the steps to follow?
Cosine Similarity Score: 0.5323439240455627
Most relevant answer: The contract review form can be found by clicking on this link. 3 versions are available:  French, English and Spanish.


c:\Users\wissa\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


GPT-2 Answer: how are you ?

A: I don't know. I think it's just a matter of time before we get to know each other. We've been friends for a long time, and we've had a lot of fun together. It's nice to be able to talk about things that are really important to you, but I'm not sure if we're going to get along as well as we would like to. But I do think that it would be nice for us to have
